In [6]:
import pandas as pd
import json
import re
from functools import partial
import datetime
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import openai
import unicodedata
f = open("openai_key.txt", "r")
openai_key = f.read()
def no_reactions(string):
    try:
        out = len(string)
    except:
        out = 0
    return out

def type_reactions(string):
    lst = []
    try:
        for item in string:
            lst.append(unicodedata.name(item['reaction']))
        tmp = [*{*lst}]
        out = ''
        for item in tmp:
            out += str(item).lower() + ','
    except:
        out = '-'
    return out

In [7]:
paths = glob.glob('/Users/ditteblom/Library/CloudStorage/OneDrive-DanmarksTekniskeUniversitet/Projekter/fb_messages/messages/inbox/gang_2039954592789078/message*.json')

df = pd.DataFrame()

for path in paths:
    fix_mojibake_escapes = partial(
        re.compile(rb'\\u00([\da-f]{2})').sub,
        lambda m: bytes.fromhex(m[1].decode()),
    )

    with open(path, 'rb') as binary_data:
        repaired = fix_mojibake_escapes(binary_data.read())
    data = json.loads(repaired)

    tmp = pd.DataFrame(data['messages'])
    df = pd.concat([df,tmp])

#df.reset_index(inplace = True, drop=True)
df['timestamp_ms'] = df['timestamp_ms'].apply(lambda x : datetime.datetime.fromtimestamp(x/1000.0))
#df.rename(columns={"timestamp_ms": "timestamp"}, inplace = True)
df['date'] = pd.to_datetime(df['timestamp_ms']).dt.date
df['time'] = pd.to_datetime(df['timestamp_ms']).dt.time
df['year'] = pd.to_datetime(df['timestamp_ms']).dt.year
df['month'] = pd.to_datetime(df['timestamp_ms']).dt.month
df['weekday'] = pd.to_datetime(df['date']).dt.day_name()
df['no_reactions'] = df.reactions.apply(lambda x: no_reactions(x))
df['type_reactions'] = df.reactions.apply(lambda x: type_reactions(x))

participants = []
for item in data['participants']:
    participants.append(item['name'])

df.sort_values(by='timestamp_ms', inplace=True)
df.reset_index(inplace = True, drop=True)

df.head(10)

,sender_name,timestamp_ms,content,call_duration,reactions,photos,share,gifs,is_unsent,sticker,videos,files,audio_files,date,time,year,month,weekday,no_reactions,type_reactions
0,Sandra Svendborg Larsen,2019-03-20 12:58:25.292,Sandra created the group.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-20,12:58:25.292000,2019,3,Wednesday,0,-
1,Sandra Svendborg Larsen,2019-03-20 12:58:44.988,Sandra named the group Gang 😏🤫.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-20,12:58:44.988000,2019,3,Wednesday,0,-
2,Sandra Svendborg Larsen,2019-03-20 12:58:59.611,Følte at vi manglede en fælles gruppe :))),NaN,"[{'reaction': '😍', 'actor': 'Josefine Harbo Ar...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-20,12:58:59.611000,2019,3,Wednesday,2,"smiling face with heart-shaped eyes,"
3,Ditte Grønborg Blom,2019-03-20 12:59:38.285,You set the quick reaction to 😈.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-20,12:59:38.285000,2019,3,Wednesday,0,-
4,Ditte Grønborg Blom,2019-03-20 12:59:57.779,You set the quick reaction to 😏.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-20,12:59:57.779000,2019,3,Wednesday,0,-
5,Ditte Grønborg Blom,2019-03-20 13:01:08.816,NaN,NaN,NaN,NaN,NaN,[{'uri': 'your_activity_across_facebook/messag...,NaN,NaN,NaN,NaN,NaN,2019-03-20,13:01:08.816000,2019,3,Wednesday,0,-
6,Louise Hjuler Andersen,2019-03-20 13:01:34.011,NaN,NaN,NaN,NaN,NaN,[{'uri': 'your_activity_across_facebook/messag...,NaN,NaN,NaN,NaN,NaN,2019-03-20,13:01:34.011000,2019,3,Wednesday,0,-
7,Nina Braad Iskov,2019-03-20 13:59:53.604,😏,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-20,13:59:53.604000,2019,3,Wednesday,0,-
8,Nina Braad Iskov,2019-03-20 14:00:12.679,Den her smiley er så vigtig,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-20,14:00:12.679000,2019,3,Wednesday,0,-
9,Josefine Harbo Arleth,2019-03-20 14:00:26.521,Jeg er så fan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-20,14:00:26.521000,2019,3,Wednesday,0,-


In [18]:
fig = px.histogram(df, x='year', color='sender_name', title='Number of sent messages per year')
fig.show()

In [19]:
fig = px.histogram(df, x='weekday', color='sender_name', title='Number of sent messages per weekday',
                   category_orders=dict(weekday=['Monday', 'Tuesday', 'Wednesday', 'Thursday','Friday','Saturday','Sunday']))
fig.show()

In [20]:
frst_date = df.sort_values(by='date').iloc[0].date
last_date = df.sort_values(by='date').iloc[-1].date
no_messages = len(df)
most_active = df.groupby(by='sender_name').count().sort_values(by='content',ascending=False).reset_index().iloc[0].sender_name
least_active = df.groupby(by='sender_name').count().sort_values(by='content',ascending=False).reset_index().iloc[-1].sender_name
most_pictures = df.groupby(by='sender_name').count().sort_values(by='photos',ascending=False).reset_index().iloc[0].sender_name
most_gifs = df.groupby(by='sender_name').count().sort_values(by='gifs',ascending=False).reset_index().iloc[0].sender_name

In [21]:
df.groupby(by='sender_name').count().sort_values(by='content',ascending=False).reset_index().iloc[0].content

6567

In [22]:
df.groupby(by='sender_name').count().sort_values(by='content',ascending=False)

,timestamp_ms,content,call_duration,reactions,photos,share,gifs,is_unsent,sticker,videos,files,audio_files,date,time,year,month,weekday,no_reactions,type_reactions
sender_name,,,,,,,,,,,,,,,,,,,
Nina Braad Iskov,6996,6567,8,1162,355,59,22,1,0,24,32,0,6996,6996,6996,6996,6996,6996,6996
Josefine Harbo Arleth,6617,6393,6,1193,152,37,48,8,0,14,2,0,6617,6617,6617,6617,6617,6617,6617
Julie van Krimpen Mortensen,5034,4862,4,674,145,15,11,2,6,2,6,0,5034,5034,5034,5034,5034,5034,5034
Ditte Grønborg Blom,4982,4674,4,1124,242,22,50,1,0,16,5,1,4982,4982,4982,4982,4982,4982,4982
Sandra Svendborg Larsen,4425,4307,0,828,88,10,20,0,0,3,8,0,4425,4425,4425,4425,4425,4425,4425
Louise Hjuler Andersen,3267,3168,2,614,94,8,1,0,1,0,4,0,3267,3267,3267,3267,3267,3267,3267


In [23]:
sad_keywords = ['crying','sad']
for keyword in sad_keywords:
    matches = df[df['type_reactions'].str.contains(keyword, case=False)]

In [24]:
matches

,sender_name,timestamp_ms,content,call_duration,reactions,photos,share,gifs,is_unsent,sticker,videos,files,audio_files,date,time,year,month,weekday,no_reactions,type_reactions


In [52]:
import numpy as np

# show sad messages
sad_df = df[df['type_reactions'].str.contains('crying')] 
idx_sad = sad_df[sad_df.no_reactions == 5].index.to_list()
if len(idx_sad) > 3:
    no_sad = 3
else:
    no_sad = len(idx_sad)
idx_sad_disp = np.random.choice(idx_sad, size = no_sad)

for val in idx_sad_disp:
    print(df.iloc[val-1].content)
    print(df.iloc[val].content)
    print(df.iloc[val+1].content)

😂😂 Hvor jeg kender det haha
Yes emil og jeg kom hjem ved 22 tiden igår aftes, men vi skulle også lige forbi riget fordi min mor er blevet indlagt igen (skete lørdag aften, men fik det først at vide ved 19-20 tiden igår aftes). Hun havde fået det skidt (mange smerter), men de ved ikke hvad der er (de kunne ikke se noget på ultralydsscanning). Men hun fik det bedre med morfin. Så nu må vi se hvor længe de holder på hende ❤️
Årh øv nins. Håber hun snart kan komme hjem igen ❤️
025 😂
Hvis I ville have opklaring, om det frygtelige vi snakkede om; (sorry for dårlig stemning!) 
Men de holdt stille på motorvejen fordi bilen var gået i stå, ham der påkørte dem lå bag en bil som skiftede vognbane i et sent øjeblik - så ham den anden når ikke at reagere og påkører dem med 110 😢😢
Åh hvor skrækkeligt altså 😢
025 😂
Hvis I ville have opklaring, om det frygtelige vi snakkede om; (sorry for dårlig stemning!) 
Men de holdt stille på motorvejen fordi bilen var gået i stå, ham der påkørte dem lå bag en bil

In [65]:
df.iloc[idx_sad_disp[1]+1]

sender_name           Sandra Svendborg Larsen
timestamp_ms       2019-12-09 18:40:59.121000
content           Åh hvor skrækkeligt altså 😢
call_duration                             NaN
reactions                                 NaN
photos                                    NaN
share                                     NaN
gifs                                      NaN
is_unsent                                 NaN
sticker                                   NaN
videos                                    NaN
files                                     NaN
audio_files                               NaN
date                               2019-12-09
time                          18:40:59.121000
year                                     2019
month                                      12
weekday                                Monday
no_reactions                                0
type_reactions                              -
Name: 5955, dtype: object

In [50]:
df.iloc[idx_sad_disp[0]+1].content

'Ej hvor irriterende Sandra :( det var nok meget godt for din krop at du tog tidligt hjem så ❤️'

In [45]:
idx_sad_disp

array([21112,  5954,  5954])

In [46]:
sad_df[sad_df.no_reactions == 5]

,sender_name,timestamp_ms,content,call_duration,reactions,photos,share,gifs,is_unsent,sticker,videos,files,audio_files,date,time,year,month,weekday,no_reactions,type_reactions
5954,Julie van Krimpen Mortensen,2019-12-09 18:34:47.932,"Hvis I ville have opklaring, om det frygtelige...",NaN,"[{'reaction': '😮', 'actor': 'Nina Braad Iskov'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-09,18:34:47.932000,2019,12,Monday,5,"crying face,face with open mouth,"
21112,Nina Braad Iskov,2021-08-30 10:44:55.888,Yes emil og jeg kom hjem ved 22 tiden igår aft...,NaN,"[{'reaction': '😢', 'actor': 'Josefine Harbo Ar...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-30,10:44:55.888000,2021,8,Monday,5,"crying face,"
22316,Sandra Svendborg Larsen,2021-10-02 08:12:06.173,Tak for igår pigerne ❤️❤️ \nEr ked af jeg gik ...,NaN,"[{'reaction': '😢', 'actor': 'Nina Braad Iskov'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-10-02,08:12:06.173000,2021,10,Saturday,5,"crying face,"
24733,Sandra Svendborg Larsen,2022-02-20 07:17:57.864,Jeg har corona piger 🥺😭❤️,NaN,"[{'reaction': '😢', 'actor': 'Nina Braad Iskov'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-02-20,07:17:57.864000,2022,2,Sunday,5,"crying face,"


In [52]:
df_sad = df[(df['type_reactions'].str.contains('crying', regex=True)) & (df['no_reactions'] > 3)]
df_sad

,sender_name,timestamp_ms,content,call_duration,reactions,photos,share,gifs,is_unsent,sticker,videos,files,audio_files,date,time,year,month,weekday,no_reactions,type_reactions
3323,Josefine Harbo Arleth,2019-09-12 21:12:41.890,Jeg tager ikke med ✌️ bare hvis I var i tvivl ...,NaN,"[{'reaction': '😢', 'actor': 'Ditte Grønborg Bl...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-09-12,21:12:41.890000,2019,9,Thursday,4,"crying face,"
5954,Julie van Krimpen Mortensen,2019-12-09 18:34:47.932,"Hvis I ville have opklaring, om det frygtelige...",NaN,"[{'reaction': '😮', 'actor': 'Nina Braad Iskov'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-09,18:34:47.932000,2019,12,Monday,5,"face with open mouth,crying face,"
6077,Sandra Svendborg Larsen,2019-12-11 20:48:10.990,NaN,NaN,"[{'reaction': '😢', 'actor': 'Nina Braad Iskov'...",[{'uri': 'your_activity_across_facebook/messag...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-11,20:48:10.990000,2019,12,Wednesday,4,"smiling face with heart-shaped eyes,crying face,"
6454,Julie van Krimpen Mortensen,2019-12-24 10:03:01.023,"Jeg smadrede min fortand i går, glædelig jul t...",NaN,"[{'reaction': '😮', 'actor': 'Nina Braad Iskov'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-24,10:03:01.023000,2019,12,Tuesday,4,"face with open mouth,crying face,"
7515,Nina Braad Iskov,2020-02-04 08:43:52.221,Jeg har også halsbetændelse og har fået det væ...,NaN,"[{'reaction': '😢', 'actor': 'Josefine Harbo Ar...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-04,08:43:52.221000,2020,2,Tuesday,4,"crying face,"
10443,Sandra Svendborg Larsen,2020-04-02 09:15:06.470,NaN,NaN,"[{'reaction': '😢', 'actor': 'Nina Braad Iskov'...",[{'uri': 'your_activity_across_facebook/messag...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-02,09:15:06.470000,2020,4,Thursday,4,"crying face,"
11749,Ditte Grønborg Blom,2020-05-09 16:40:39.441,NaN,NaN,"[{'reaction': '😢', 'actor': 'Nina Braad Iskov'...",[{'uri': 'your_activity_across_facebook/messag...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-05-09,16:40:39.441000,2020,5,Saturday,4,"crying face,"
14711,Nina Braad Iskov,2020-08-08 09:14:46.075,Hey piger. Vil bare lige fortælle at Emil og j...,NaN,"[{'reaction': '😢', 'actor': 'Ditte Grønborg Bl...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-08-08,09:14:46.075000,2020,8,Saturday,4,"crying face,"
14850,Julie van Krimpen Mortensen,2020-08-18 08:37:34.432,Jeg ligger desværre syg i dag 😭 Har også meldt...,NaN,"[{'reaction': '😢', 'actor': 'Nina Braad Iskov'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-08-18,08:37:34.432000,2020,8,Tuesday,4,"crying face,"
15013,Louise Hjuler Andersen,2020-08-26 15:05:36.787,NaN,NaN,"[{'reaction': '😢', 'actor': 'Josefine Harbo Ar...",[{'uri': 'your_activity_across_facebook/messag...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-08-26,15:05:36.787000,2020,8,Wednesday,4,"crying face,"


In [48]:
tmp.iloc[2].photos

[{'uri': 'your_activity_across_facebook/messages/inbox/gang_2039954592789078/photos/79105230_2506153192839316_2815154468989960192_n_1196237253910267.jpg',
  'creation_timestamp': 1576093690}]

In [33]:
df.iloc[idx_sad_disp[0]]

sender_name                 Nina Braad Iskov
timestamp_ms      2019-04-11 10:17:36.424000
content                               oki <3
call_duration                            NaN
reactions                                NaN
photos                                   NaN
share                                    NaN
gifs                                     NaN
is_unsent                                NaN
sticker                                  NaN
videos                                   NaN
files                                    NaN
audio_files                              NaN
date                              2019-04-11
time                         10:17:36.424000
year                                    2019
month                                      4
weekday                             Thursday
no_reactions                               0
type_reactions                             -
Name: 21112, dtype: object

In [8]:
from langchain.agents import create_pandas_dataframe_agent
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain.llms import OpenAI

#agent = create_pandas_dataframe_agent(OpenAI(temperature=0, openai_api_key = openai_key), df, verbose=True)
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613", openai_api_key = openai_key),
    df,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [31]:
agent.run('Kan du give mig indeces på de fem beskeder som virker sjovest')



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['content'].str.contains('😂|🤣|😆|😅|😄', regex=True)"}`


0        False
1        False
2        False
3        False
4        False
         ...  
31316    False
31317    False
31318    False
31319    False
31320    False
Name: content, Length: 31321, dtype: object
Invoking: `python_repl_ast` with `{'query': "df[df['content'].str.contains('😂|🤣|😆|😅|😄', regex=True)].head(5).index.tolist()"}`


ValueError: Cannot mask with non-boolean array containing NA / NaN values
Invoking: `python_repl_ast` with `{'query': "df[df['content'].str.contains('😂|🤣|😆|😅|😄', regex=True, na=False)].head(5).index.tolist()"}`


[18, 20, 29, 74, 76]

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo-0613 in organization org-XxONlHXzDNvYgVbzjS0c47fJ on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..


De fem beskeder, der virker sjovest, har følgende indeks i dataframe:
- 18
- 20
- 29
- 74
- 76

> Finished chain.


'De fem beskeder, der virker sjovest, har følgende indeks i dataframe:\n- 18\n- 20\n- 29\n- 74\n- 76'

In [34]:
df.iloc[20]

sender_name                     Ditte Grønborg Blom
timestamp_ms             2019-04-01 19:08:30.226000
content           Jeg skal i hvert fald have noget😅
call_duration                                   NaN
reactions                                       NaN
photos                                          NaN
share                                           NaN
gifs                                            NaN
is_unsent                                       NaN
sticker                                         NaN
videos                                          NaN
files                                           NaN
audio_files                                     NaN
date                                     2019-04-01
time                                19:08:30.226000
year                                           2019
month                                             4
weekday                                      Monday
no_reactions                                      0
type_reactio

In [56]:
import numpy as np
df_sad = df[(df['type_reactions'].str.contains('crying', regex=True)) & (df['no_reactions'] > 3)]
idx_sad = df_sad[df_sad.no_reactions > 3].index.to_list()
if len(idx_sad) > 5:
    no_sad = 5
else:
    no_sad = len(idx_sad)
idx_sad_disp = np.random.choice(idx_sad, size = no_sad)

In [66]:
for val in idx_sad_disp:
    print('index')
    print(df.iloc[val-1].time)
    print(df.iloc[val].time)
    print(df.iloc[val+1].time)

index
08:35:51.365000
08:37:34.432000
08:37:59.152000
index
09:13:23.495000
09:15:06.470000
09:15:10.192000
index
23:19:50.584000
09:17:47.344000
20:34:45.853000
index
08:35:51.365000
08:37:34.432000
08:37:59.152000
index
08:42:33.606000
18:34:47.932000
18:40:59.121000


In [70]:
df.iloc[val].time - datetime.timedelta(minutes = 5)

TypeError: unsupported operand type(s) for -: 'datetime.time' and 'datetime.timedelta'

In [76]:
type(df.iloc[val].time) 

datetime.time

In [80]:
df.iloc[val].timestamp_ms - pd.Timedelta(minutes=5)

Timestamp('2019-12-09 18:29:47.932000')

In [59]:
for val in idx_sad_disp:
    print('index')
    print(df.iloc[val-1].content)
    print(df.iloc[val].content)
    print(df.iloc[val+1].content)

index
Hvor tænker i at vi skal spise henne i dag @Nina Braad Iskov @Julie van Krimpen Mortensen @Josefine Harbo Arleth? ❤️
Jeg ligger desværre syg i dag 😭 Har også meldt mig syg fra arbejdet, så jeg kommer desværre ikke i dag 😭😭❤️
Øv hvor ærgerligt Jules, god bedring ❤️❤️
index
Jaaa virkelig, ville gerne have den ud af verdenen 😍
nan
:(((
index
Det var så godt at se jer piger!!😘😘❤️
Ift. advanced ML kan godt se der står deltagerbegrænsning på 100, og pt er der 220 tilmeldte :((
Så har jeg haft første teoritime og skal til manøvrebane fra 7-10 på søndag på Amager 😅✌🏼
index
Hvor tænker i at vi skal spise henne i dag @Nina Braad Iskov @Julie van Krimpen Mortensen @Josefine Harbo Arleth? ❤️
Jeg ligger desværre syg i dag 😭 Har også meldt mig syg fra arbejdet, så jeg kommer desværre ikke i dag 😭😭❤️
Øv hvor ærgerligt Jules, god bedring ❤️❤️
index
025 😂
Hvis I ville have opklaring, om det frygtelige vi snakkede om; (sorry for dårlig stemning!) 
Men de holdt stille på motorvejen fordi bilen var 

In [65]:
df

,sender_name,timestamp_ms,content,call_duration,reactions,photos,share,gifs,is_unsent,sticker,videos,files,audio_files,date,time,year,month,weekday,no_reactions,type_reactions
0,Sandra Svendborg Larsen,2019-03-20 12:58:25.292,Sandra created the group.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-20,12:58:25.292000,2019,3,Wednesday,0,-
1,Sandra Svendborg Larsen,2019-03-20 12:58:44.988,Sandra named the group Gang 😏🤫.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-20,12:58:44.988000,2019,3,Wednesday,0,-
2,Sandra Svendborg Larsen,2019-03-20 12:58:59.611,Følte at vi manglede en fælles gruppe :))),NaN,"[{'reaction': '😍', 'actor': 'Josefine Harbo Ar...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-20,12:58:59.611000,2019,3,Wednesday,2,"smiling face with heart-shaped eyes,"
3,Ditte Grønborg Blom,2019-03-20 12:59:38.285,You set the quick reaction to 😈.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-20,12:59:38.285000,2019,3,Wednesday,0,-
4,Ditte Grønborg Blom,2019-03-20 12:59:57.779,You set the quick reaction to 😏.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-20,12:59:57.779000,2019,3,Wednesday,0,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31316,Josefine Harbo Arleth,2023-09-16 14:27:31.108,Hej girls. Jeg holder min fødselsdag på dagen ...,NaN,"[{'reaction': '❤', 'actor': 'Nina Braad Iskov'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-09-16,14:27:31.108000,2023,9,Saturday,3,"heavy black heart,"
31317,Sandra Svendborg Larsen,2023-09-16 14:30:46.658,"Jaaaa, elsker Jose-fødselsdage 😍😍 Det er skrev...",NaN,"[{'reaction': '❤', 'actor': 'Josefine Harbo Ar...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-09-16,14:30:46.658000,2023,9,Saturday,1,"heavy black heart,"
31318,Nina Braad Iskov,2023-09-16 15:54:33.440,Yay me too 😍😍,NaN,"[{'reaction': '❤', 'actor': 'Josefine Harbo Ar...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-09-16,15:54:33.440000,2023,9,Saturday,1,"heavy black heart,"
31319,Ditte Grønborg Blom,2023-09-16 16:21:54.664,Jeg kan desværre ikke 😭😭❤️,NaN,"[{'reaction': '😢', 'actor': 'Josefine Harbo Ar...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-09-16,16:21:54.664000,2023,9,Saturday,1,"crying face,"
